In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql, extras
from psycopg2.extras import execute_values

from dotenv import load_dotenv
import os
import pandas as pd

In [8]:
load_dotenv()

# Datos de conexión
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

# Ruta al CSV
csv_file = '/Users/marcelomiqueljuan/Documents/Projects/My Town Weather Analysis/Input/st_antoni.csv'  # ⚡ pon la ruta correcta aquí

# Leer el CSV
df = pd.read_csv(csv_file)


In [291]:
df['dt_iso']


0         1979-01-01 00:00:00 +0000 UTC
1         1979-01-01 01:00:00 +0000 UTC
2         1979-01-01 02:00:00 +0000 UTC
3         1979-01-01 03:00:00 +0000 UTC
4         1979-01-01 04:00:00 +0000 UTC
                      ...              
405427    2025-04-01 19:00:00 +0000 UTC
405428    2025-04-01 20:00:00 +0000 UTC
405429    2025-04-01 21:00:00 +0000 UTC
405430    2025-04-01 22:00:00 +0000 UTC
405431    2025-04-01 23:00:00 +0000 UTC
Name: dt_iso, Length: 405432, dtype: object

In [292]:
# Step 1: Remove the ' UTC' part from the datetime strings
df['dt_iso'] = df['dt_iso'].str.replace(' UTC', '', regex=False)

# Step 2: Convert the cleaned string to datetime with UTC timezone
df['dt_iso'] = pd.to_datetime(df['dt_iso'], utc=True)



In [293]:

# Mapeo para que pandas coincida con la tabla
df = df.rename(columns={
    'city_name': 'city_name',
    'lat': 'lat',
    'lon': 'lon',
    'temp': 'temp',
    'temp_min': 'temp_min',
    'temp_max': 'temp_max',
    'feels_like': 'feels_like',
    'pressure': 'pressure',
    'humidity': 'humidity',
    'dew_point': 'dew_point',
    'wind_speed': 'wind_speed',
    'wind_deg': 'wind_deg',
    'wind_gust': 'wind_gust',
    'clouds_all': 'cloudiness',
    'rain_1h': 'rain_1h',
    'rain_3h': 'rain_3h',
    'snow_1h': 'snow_1h',
    'snow_3h': 'snow_3h',
    'weather_id': 'weather_id',
    'weather_main': 'weather_main',
    'weather_description': 'weather_description',
    'weather_icon': 'weather_icon',
    'visibility': 'visibility',
    'dt': 'dt',
    'dt_iso': 'dt_iso',
    'timezone': 'timezone'
})

# Solo las columnas que tenemos en la tabla
expected_cols = [
    'city_name', 'lat', 'lon', 'temp', 'temp_min', 'temp_max', 'feels_like',
    'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg', 'wind_gust',
    'cloudiness', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'weather_id',
    'weather_main', 'weather_description', 'weather_icon', 'visibility',
    'dt', 'dt_iso', 'timezone'
]
df = df[expected_cols]

# Conexión a la base de datos
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)



In [294]:

cur = conn.cursor()
cur.execute('SET search_path TO public')
# Prepare list of tuples
data = [tuple(row) for row in df[expected_cols].values]


In [295]:
cur = conn.cursor()
cur.execute('SELECT current_database();')
print('👉 Connected to:', cur.fetchone())

👉 Connected to: ('weather_project_db',)


In [296]:
# Prepare the insert statement
insert_query = sql.SQL("""
    INSERT INTO stg_weather_data ({fields})
    VALUES ({placeholders})
""").format(
    fields=sql.SQL(', ').join(map(sql.Identifier, expected_cols)),
    placeholders=sql.SQL(', ').join(sql.Placeholder() * len(expected_cols))
)

# Clean and prepare data
df = df[expected_cols].where(pd.notnull(df), None)
df['visibility'] = df['visibility'].apply(lambda x: int(x) if pd.notnull(x) else None)

# Convert the dataframe to a list of tuples
data_tuples = [tuple(x) for x in df[expected_cols].to_numpy()]

# Batch insert setup
batch_size = 5000

for i in range(0, len(data_tuples), batch_size):
    batch = data_tuples[i:i + batch_size]
    try:
        extras.execute_batch(cur, insert_query, batch)
        conn.commit()
        print(f"✅ Inserted batch {i // batch_size + 1}: rows {i} to {i + len(batch) - 1}")
    except Exception as e:
        print(f"❌ Error inserting batch starting at row {i}")
        print("Error message:", e)
        conn.rollback()
        break  # You can change this to continue if you want to skip errors

cur.close()
conn.close()

print("✅ All batches processed successfully")


✅ Inserted batch 1: rows 0 to 4999
✅ Inserted batch 2: rows 5000 to 9999
✅ Inserted batch 3: rows 10000 to 14999
✅ Inserted batch 4: rows 15000 to 19999
✅ Inserted batch 5: rows 20000 to 24999
✅ Inserted batch 6: rows 25000 to 29999
✅ Inserted batch 7: rows 30000 to 34999
✅ Inserted batch 8: rows 35000 to 39999
✅ Inserted batch 9: rows 40000 to 44999
✅ Inserted batch 10: rows 45000 to 49999
✅ Inserted batch 11: rows 50000 to 54999
✅ Inserted batch 12: rows 55000 to 59999
✅ Inserted batch 13: rows 60000 to 64999
✅ Inserted batch 14: rows 65000 to 69999
✅ Inserted batch 15: rows 70000 to 74999
✅ Inserted batch 16: rows 75000 to 79999
✅ Inserted batch 17: rows 80000 to 84999
✅ Inserted batch 18: rows 85000 to 89999
✅ Inserted batch 19: rows 90000 to 94999
✅ Inserted batch 20: rows 95000 to 99999
✅ Inserted batch 21: rows 100000 to 104999
✅ Inserted batch 22: rows 105000 to 109999
✅ Inserted batch 23: rows 110000 to 114999
✅ Inserted batch 24: rows 115000 to 119999
✅ Inserted batch 25: row

In [298]:
data_tuples

[('Sant Antoni',
  38.553135,
  -0.506159,
  13.21,
  11.59,
  14.23,
  12.31,
  1017,
  66,
  7.0,
  4.72,
  276,
  nan,
  13,
  nan,
  nan,
  nan,
  nan,
  801,
  'Clouds',
  'few clouds',
  '02n',
  None,
  283996800,
  Timestamp('1979-01-01 00:00:00+0000', tz='UTC'),
  3600),
 ('Sant Antoni',
  38.553135,
  -0.506159,
  13.25,
  11.75,
  14.22,
  12.46,
  1016,
  70,
  7.9,
  4.76,
  277,
  nan,
  14,
  nan,
  nan,
  nan,
  nan,
  801,
  'Clouds',
  'few clouds',
  '02n',
  None,
  284000400,
  Timestamp('1979-01-01 01:00:00+0000', tz='UTC'),
  3600),
 ('Sant Antoni',
  38.553135,
  -0.506159,
  13.37,
  11.88,
  14.41,
  12.62,
  1016,
  71,
  8.23,
  4.21,
  275,
  nan,
  26,
  nan,
  nan,
  nan,
  nan,
  802,
  'Clouds',
  'scattered clouds',
  '03n',
  None,
  284004000,
  Timestamp('1979-01-01 02:00:00+0000', tz='UTC'),
  3600),
 ('Sant Antoni',
  38.553135,
  -0.506159,
  12.25,
  10.07,
  13.45,
  11.33,
  1016,
  69,
  6.74,
  3.92,
  276,
  nan,
  46,
  nan,
  nan,
  nan,
